In [51]:
import __init__
import os
import numpy as np
import pandas as pd
import yaml
import joblib
import lightgbm as lgb
from tqdm import tqdm
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import roc_auc_score, roc_curve, balanced_accuracy_score
import matplotlib.pyplot as plt
from datapath_manager import DataPathManager, ITWDataPathManager
from trainers import MachineLearningModelTrainer, BranchNeuralNetworkTrainer
from dataloader import EmbeddingDataLoader
from evaluators import Evaluator

In [101]:
# user_id = 'ltkhiem'
# dates = ['2022-10-16', '2022-10-17', '2022-10-18', '2022-10-20', '2022-10-21', '2022-10-22']
user_id = 'lzhou'
dates = ['2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15', '2022-09-16']
# user_id = 'tlduyen'
# dates = ['2022-09-27', '2022-09-28', '2022-09-30', '2022-10-01', '2022-10-02', '2022-10-03']

In [53]:
model_path = os.path.abspath(f'./models/{user_id}')

# Lab-based model for the targeted user

In [54]:
lab_dataset_name = 'DCU_NVT_EXP2'
window_size = 60
window_shift = 0.25
signal_type = 'bvp_eda_temp'

In [55]:
dp_manager = DataPathManager(lab_dataset_name)
lab_feature_folder_path = os.path.dirname(dp_manager.get_feature_path(user_id, signal_type, window_size, window_shift))
ground_truth_path = os.path.join(lab_feature_folder_path, 'ground_truth.npy')
tasks_index_path = os.path.join(lab_feature_folder_path, 'tasks_index.npy')

In [56]:
lab_features = []
signals = ['bvp', 'eda', 'temp']
for signal in signals:
    signal_path = os.path.join(lab_feature_folder_path, f'{signal}.npy')
    signal_data = np.load(signal_path)
    lab_features.append(signal_data)
lab_features = np.concatenate(lab_features, axis=1)
tasks_index = np.load(tasks_index_path)
ground_truth = np.load(ground_truth_path) 

In [57]:
# Split train/test lab-based model
def split_train_test(indices, test_size: float = 0.3):
        """
        Split train and test data for subject-dependent model training:
            - Train_data: (1 - test_size) * number of data of a class
            - Test_data: test_size * number of data of a class 
        NOTE: This means that this approach of data splitting simulate the real-life situation 
        where the test data is the segment of data that is recorded later after we have the train data.
        """
        cut_point = int((1 - test_size) * len(indices))
        train_indices = indices[:cut_point].tolist()
        test_indices = indices[cut_point:].tolist()
        return train_indices, test_indices


TEST_SIZE = 0.2
VALID_SIZE = 0.1
indices = np.arange(lab_features.shape[0]) # The indices of the lab-based features
train_indices, valid_indices, test_indices = [], [], []
for _, task_test_index in LeaveOneGroupOut().split(indices, y=None, groups=tasks_index):
    task_train_indices, task_test_indices = split_train_test(indices[task_test_index], test_size = TEST_SIZE)
    task_train_indices, task_valid_indices = split_train_test(indices[task_train_indices], test_size = VALID_SIZE)
    train_indices += task_train_indices
    valid_indices += task_valid_indices
    test_indices += task_test_indices

### 1. Train Deep-Fusion Model

In [58]:
X_train, y_train = lab_features[train_indices], ground_truth[train_indices]
X_valid, y_valid = lab_features[valid_indices], ground_truth[valid_indices]
X_test, y_test = lab_features[test_indices], ground_truth[test_indices]

In [59]:
train_dataloader = EmbeddingDataLoader(X_train, y_train)
validate_dataloader = EmbeddingDataLoader(X_valid, y_valid)
test_dataloader = EmbeddingDataLoader(X_test, y_test)

In [60]:
# Load deep model configuration
user_model_saved_path = os.path.join(f'{model_path}/lab_deep_fusion.pth')
config_path = os.path.join(
    os.path.dirname(os.getcwd()), 
    'models', 'model_config', 
    f'branchnn_sensor_combination_{signal_type}.yaml'
)
config_dict = yaml.safe_load(open(config_path, 'r'))

In [61]:
df_clf = BranchNeuralNetworkTrainer('./logs.txt', 
    user_model_saved_path, 
    config_dict, 
    target_metrics=['balanced_accuracy', 'f1'],
)

LOAD PRETRAINED MODEL


In [62]:
# df_clf.train(train_dataloader, validate_dataloader, num_epochs=1000)

In [63]:
y_pred = df_clf.predict(test_dataloader)
print(Evaluator().evaluate(y_test, y_pred))

{'accuracy': 0.7399913904433921, 'balanced_accuracy': 0.6873414337788578, 'precision': 0.7622950819672131, 'recall': 0.868, 'f1': 0.8117206982543641}


### 2. Train ExtraTreesClassifier Model

In [64]:
# X_train = np.concatenate([lab_features[train_indices], lab_features[valid_indices]], axis=0)
# # y_train = np.concatenate([ground_truth[train_indices], ground_truth[valid_indices]], axis=0)
# X_test, y_test = lab_features[test_indices], ground_truth[test_indices]

et_clf = ExtraTreesClassifier(
    n_estimators = 500,
    random_state = 0, 
    n_jobs = -1, 
    max_features = 'sqrt', 
    max_depth = 8, 
    min_samples_split = 2, 
    min_samples_leaf = 8,
    oob_score = True, 
    bootstrap = True, 
    class_weight = 'balanced'
)
et_clf.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=True, class_weight='balanced', max_depth=8,
                     min_samples_leaf=8, n_estimators=500, n_jobs=-1,
                     oob_score=True, random_state=0)

In [65]:
y_pred = et_clf.predict(X_valid)
print(Evaluator().evaluate(y_valid, y_pred))

{'accuracy': 0.9817204301075269, 'balanced_accuracy': 0.9858333333333333, 'precision': 1.0, 'recall': 0.9716666666666667, 'f1': 0.9856297548605241}


In [66]:
y_pred = et_clf.predict(X_test)
print(Evaluator().evaluate(y_test, y_pred))

{'accuracy': 0.8932414980628498, 'balanced_accuracy': 0.8858003240178209, 'precision': 0.9224021592442645, 'recall': 0.9113333333333333, 'f1': 0.9168343393695507}


In [67]:
data = {'model': et_clf, 'scaler': None}
joblib.dump(data, os.path.join(model_path, 'lab_et_clf.pkl'))

['/home/nvtu/Stress-Detection-Processor/experiment_scripts/models/ltkhiem/lab_et_clf.pkl']

### 3. Train Logistic Regression Model

In [68]:
std_scaler = StandardScaler()
X_train = std_scaler.fit_transform(np.concatenate([lab_features[train_indices], lab_features[valid_indices]], axis=0))
y_train = np.concatenate([ground_truth[train_indices], ground_truth[valid_indices]], axis=0)
X_valid, y_valid = std_scaler.transform(lab_features[valid_indices]), ground_truth[valid_indices]
X_test, y_test = std_scaler.transform(lab_features[test_indices]), ground_truth[test_indices]

lr_clf = LogisticRegression(
    random_state = 0,
    class_weight = 'balanced',
    n_jobs = -1,
    solver = 'saga',
    max_iter = 50000,
)
lr_clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=50000, n_jobs=-1,
                   random_state=0, solver='saga')

In [69]:
y_pred = lr_clf.predict(X_valid)
print(Evaluator().evaluate(y_valid, y_pred))

{'accuracy': 0.9978494623655914, 'balanced_accuracy': 0.9983333333333333, 'precision': 1.0, 'recall': 0.9966666666666667, 'f1': 0.9983305509181971}


In [70]:
y_pred = lr_clf.predict(X_test)
print(Evaluator().evaluate(y_test, y_pred))

{'accuracy': 0.7619457597933706, 'balanced_accuracy': 0.6983090319967598, 'precision': 0.7626178591236827, 'recall': 0.9166666666666666, 'f1': 0.8325764456554647}


In [71]:
data = {'model': lr_clf, 'scaler': std_scaler}
joblib.dump(data, os.path.join(model_path, 'lab_lr_clf.pkl'))

['/home/nvtu/Stress-Detection-Processor/experiment_scripts/models/ltkhiem/lab_lr_clf.pkl']

# In-the-wild model for the targeted user

In [72]:
itw_dataset_name = 'DCU_EXP2_ITW'

In [73]:
def get_features_and_labels(user_id: str, date: str,):
    dataset_path = ITWDataPathManager(itw_dataset_name).get_dataset_path()

    user_date_feature_path = os.path.join(dataset_path, 'features', user_id, date)
    feature_path = os.path.join(user_date_feature_path, 'X.npy')
    gt_path = os.path.join(user_date_feature_path, 'y.npy')

    feat = np.nan_to_num(np.load(feature_path))[:, :72]
    gt = np.load(gt_path)
    return feat, gt

In [74]:
def get_moments_features_and_labels(user_id: str, date: str,):
    dataset_path = ITWDataPathManager(itw_dataset_name).get_dataset_path()

    user_date_feature_path = os.path.join(dataset_path, 'features', user_id, date)
    feature_path = os.path.join(user_date_feature_path, 'X_moment.npy')
    gt_path = os.path.join(user_date_feature_path, 'y_moment.npy')

    feat = np.nan_to_num(np.load(feature_path))[:, :72]
    gt = np.load(gt_path)
    return feat, gt

Counter({0.0: 16619, 1.0: 1242})
Counter({0.0: 30000, 1.0: 7224})
Counter({0.0: 39579, 1.0: 6050})
Counter({0.0: 48606})
Counter({0.0: 23292, 1.0: 5447})
Counter({0.0: 7408, 1.0: 2120})


In [75]:
# Load data
feat_dates = [get_features_and_labels(user_id, date) for date in dates]
for x in feat_dates:
    print(Counter(x[1]))

Counter({0.0: 16619, 1.0: 1242})
Counter({0.0: 30000, 1.0: 7224})
Counter({0.0: 39579, 1.0: 6050})
Counter({0.0: 48606})
Counter({0.0: 23292, 1.0: 5447})
Counter({0.0: 7408, 1.0: 2120})


In [102]:
# Load data
moments_feat_dates = [get_moments_features_and_labels(user_id, date) for date in dates]
for x in moments_feat_dates:
    print(Counter(x[1]))

Counter({0: 642, 1: 49})
Counter({1: 513, 0: 125})
Counter({1: 780, 0: 199})
Counter({0: 331, 1: 300})
Counter({1: 563, 0: 501})
Counter({1: 166, 0: 100})


## 1. Apply lab-based model to in-the-wild data

In [77]:
X_test_itw = np.concatenate([x[0] for x in feat_dates], axis=0)
y_test_itw = np.concatenate([x[1] for x in feat_dates], axis=0)
X_test_moment_itw = np.concatenate([x[0] for x in moments_feat_dates], axis=0)
y_test_moment_itw = np.concatenate([x[1] for x in moments_feat_dates], axis=0)

In [78]:
itw_test_dataloader = EmbeddingDataLoader(X_test_itw, y_test_itw)
itw_moment_test_dataloader = EmbeddingDataLoader(X_test_moment_itw, y_test_moment_itw)

In [79]:
itw_user_model_saved_path = os.path.abspath(f'{model_path}/itw_deep_fusion.pth')
itw_df_clf = BranchNeuralNetworkTrainer('./logs.txt',
    itw_user_model_saved_path,
    config_dict,
    target_metrics=['balanced_accuracy', 'f1'],
    pretrained_model_path = user_model_saved_path
)
itw_et_clf = joblib.load(os.path.join(model_path, 'lab_et_clf.pkl'))['model']
itw_lr_clf = joblib.load(os.path.join(model_path, 'lab_lr_clf.pkl'))['model']
std_scaler = joblib.load(os.path.join(model_path, 'lab_lr_clf.pkl'))['scaler']

LOAD PRETRAINED MODEL


In [80]:
# Lab-based Deep Fusion model applied to ITW data
print("--- Deep Fusion ---")
y_pred_itw = itw_df_clf.predict(itw_test_dataloader)
print(Evaluator().evaluate(y_test_itw, y_pred_itw))
print("--- Extra Trees ---")
y_pred_itw = itw_et_clf.predict(X_test_itw)
print(Evaluator().evaluate(y_test_itw, y_pred_itw))
print("--- Logistic Regression ---")
y_pred_itw = itw_lr_clf.predict(std_scaler.transform(X_test_itw))
print(Evaluator().evaluate(y_test_itw, y_pred_itw))

--- Deep Fusion ---
{'accuracy': 0.6056762995303513, 'balanced_accuracy': 0.5728674240878092, 'precision': 0.15543277595526808, 'recall': 0.5299551691346285, 'f1': 0.24036723627998688}
--- Extra Trees ---
{'accuracy': 0.5223762840708578, 'balanced_accuracy': 0.5947644795360743, 'precision': 0.1554158202586691, 'recall': 0.6894443689716071, 'f1': 0.25365276643953155}
--- Logistic Regression ---
{'accuracy': 0.6167271719255599, 'balanced_accuracy': 0.5744603678724267, 'precision': 0.15760750027493675, 'recall': 0.5191776479644976, 'f1': 0.24180876754510847}


In [81]:
# Lab-based Deep Fusion model applied to ITW data
print("--- Deep Fusion ---")
y_pred_itw = itw_df_clf.predict(itw_moment_test_dataloader)
print(Evaluator().evaluate(y_test_moment_itw, y_pred_itw))
print("--- Extra Trees ---")
y_pred_itw = itw_et_clf.predict(X_test_moment_itw)
print(Evaluator().evaluate(y_test_moment_itw, y_pred_itw))
print("--- Logistic Regression ---")
y_pred_itw = itw_lr_clf.predict(std_scaler.transform(X_test_moment_itw))
print(Evaluator().evaluate(y_test_moment_itw, y_pred_itw))

--- Deep Fusion ---
{'accuracy': 0.553025641025641, 'balanced_accuracy': 0.5581207989146115, 'precision': 0.2297003907946157, 'recall': 0.5663811563169164, 'f1': 0.3268458449181341}
--- Extra Trees ---
{'accuracy': 0.4886153846153846, 'balanced_accuracy': 0.5713830661790316, 'precision': 0.22905804657629475, 'recall': 0.7055674518201285, 'f1': 0.3458409866176856}
--- Logistic Regression ---
{'accuracy': 0.558974358974359, 'balanced_accuracy': 0.5589408442622906, 'precision': 0.23097345132743363, 'recall': 0.5588865096359743, 'f1': 0.3268628678772699}


## 2. Fine-tune the lab-based model to adapt to the in-the-wild data

In [82]:
# Use the first 3 days for training
X_train = np.concatenate([x[0] for x in feat_dates[:3]])
y_train = np.concatenate([x[1] for x in feat_dates[:3]]).astype(int)
# Use the last 3 days for testing
X_test = np.concatenate([x[0] for x in feat_dates[3:]])
y_test = np.concatenate([x[1] for x in feat_dates[3:]]).astype(int)

### Fine-tune Deep-Fusion Model

In [83]:
itw_train_dataloader = EmbeddingDataLoader(X_train, y_train)
itw_test_dataloader = EmbeddingDataLoader(X_test, y_test)

In [84]:
itw_df_clf = BranchNeuralNetworkTrainer('./logs.txt', 
    itw_user_model_saved_path, 
    config_dict, 
    target_metrics=['balanced_accuracy', 'f1'],
)

LOAD PRETRAINED MODEL


In [85]:
# itw_df_clf.train(itw_train_dataloader, itw_test_dataloader, num_epochs=1000)

{'accuracy': 0.7303650155974813, 'balanced_accuracy': 0.6183724672509305, 'precision': 0.15770841428139706, 'recall': 0.48275406369763446, 'f1': 0.237748128864302}


In [86]:
y_pred_itw = itw_df_clf.predict(itw_test_dataloader)
print(Evaluator().evaluate(y_test, y_pred_itw))

{'accuracy': 0.7358788115985404, 'balanced_accuracy': 0.6280868447019602, 'precision': 0.1643530644316396, 'recall': 0.4975551737808907, 'f1': 0.24708777686628386}


## 3. Re-train ML Model

### Re-train ExtraTreesClassifier Model

In [87]:
itw_et_clf = ExtraTreesClassifier(
    n_estimators = 500,
    random_state = 0, 
    n_jobs = -1, 
    max_features = 'sqrt', 
    max_depth = 8, 
    min_samples_split = 2, 
    min_samples_leaf = 8,
    oob_score = True, 
    bootstrap = True, 
    class_weight = 'balanced'
)
itw_et_clf.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=True, class_weight='balanced', max_depth=8,
                     min_samples_leaf=8, n_estimators=500, n_jobs=-1,
                     oob_score=True, random_state=0)

In [88]:
y_pred_itw = itw_et_clf.predict(X_test)
print(Evaluator().evaluate(y_test, y_pred_itw))

{'accuracy': 0.6796933454583127, 'balanced_accuracy': 0.6655729083471642, 'precision': 0.16316962058989792, 'recall': 0.6484736355226642, 'f1': 0.26073326248671624}


In [89]:
data = {'model': itw_et_clf, 'scaler': None}
joblib.dump(data, os.path.join(model_path, 'itw_et_clf.pkl'))

['/home/nvtu/Stress-Detection-Processor/experiment_scripts/models/ltkhiem/itw_et_clf.pkl']

### Re-train Logistic Regression Model

In [90]:
scaler = StandardScaler()

In [91]:
itw_lr_clf = LogisticRegression(
    random_state = 0,
    class_weight = 'balanced',
    n_jobs = -1,
    solver = 'saga',
    max_iter = 1000,
)
itw_lr_clf.fit(scaler.fit_transform(X_train), y_train)

/home/nvtu/.conda/envs/huawei_nvtu/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1,
                   random_state=0, solver='saga')

In [92]:
y_pred_itw = itw_lr_clf.predict(scaler.transform(X_test))
print(Evaluator().evaluate(y_test, y_pred_itw))

{'accuracy': 0.7397119933696316, 'balanced_accuracy': 0.6241493667423496, 'precision': 0.16376882849863675, 'recall': 0.48420774415224, 'f1': 0.2447561790247161}


In [93]:
data = {'model': itw_lr_clf, 'scaler': std_scaler}
joblib.dump(data, os.path.join(model_path, 'itw_lr_clf.pkl'))

['/home/nvtu/Stress-Detection-Processor/experiment_scripts/models/ltkhiem/itw_lr_clf.pkl']

## 4. Evaluate the in-the-wild model with lifelog image moments

In [94]:
# Use the first 3 days for training
X_train_moments = np.concatenate([x[0] for x in moments_feat_dates[:3]])
y_train_moments = np.concatenate([x[1] for x in moments_feat_dates[:3]]).astype(int)
# Use the last 3 days for testing
X_test_moments = np.concatenate([x[0] for x in moments_feat_dates[3:]])
y_test_moments = np.concatenate([x[1] for x in moments_feat_dates[3:]]).astype(int)

In [95]:
mitw_train_dataloader = EmbeddingDataLoader(X_train_moments, y_train_moments)
mitw_test_dataloader = EmbeddingDataLoader(X_test_moments, y_test_moments)

In [96]:
y_preds = itw_df_clf.predict(mitw_test_dataloader)
# y_preds = np.where(y_preds > df_thresholdOpt, 1, 0)
print(Evaluator().evaluate(y_test_moments, y_preds))

{'accuracy': 0.659427966101695, 'balanced_accuracy': 0.623865233947255, 'precision': 0.21994134897360704, 'recall': 0.5747126436781609, 'f1': 0.3181336161187699}


In [97]:
y_preds = itw_et_clf.predict(X_test_moments)
# y_preds = np.where(y_preds > thresholdOpt, 1, 0)
print(Evaluator().evaluate(y_test_moments, y_preds))

{'accuracy': 0.576271186440678, 'balanced_accuracy': 0.6383443189284277, 'precision': 0.20610687022900764, 'recall': 0.7241379310344828, 'f1': 0.3208828522920204}


## 5. Ensemble the models

In [98]:
df_preds_prob = itw_df_clf.predict_proba(mitw_test_dataloader)
et_preds_prob = itw_et_clf.predict_proba(X_test_moments)[:, 1]

In [99]:
preds_prob = 0.05 * np.array(df_preds_prob) + 0.95 * np.array(et_preds_prob)
preds = np.where(preds_prob > 0.5, 1, 0)

In [100]:
print(Evaluator().evaluate(y_test_moments, preds))

{'accuracy': 0.5911016949152542, 'balanced_accuracy': 0.6501659025025491, 'precision': 0.21388577827547592, 'recall': 0.7318007662835249, 'f1': 0.3310225303292894}
